# CodeReviewer Tutorial

The **CodeReviewer** agent reviews code for quality, security, and best practices.

## Overview

- **Type**: Simple Agent (single input/output)
- **Use Case**: Code review, security scanning, style checking
- **Features**: Presets for different review focuses, customizable criteria

## Setup

In [ ]:
from agent_workshop.agents.software_dev import CodeReviewer, get_preset, list_presets
from agent_workshop import Config

config = Config()
reviewer = CodeReviewer(config)

print(f"Provider: {reviewer.provider_name}")
print(f"Model: {reviewer.model_name}")

## Available Presets

CodeReviewer comes with pre-configured review focuses:

In [ ]:
# List all available presets
print("Available Presets:")
print("=" * 50)
for preset in list_presets():
    print(f"\n{preset['name']}:")
    print(f"  {preset['description']}")

In [ ]:
# Using a specific preset
security_preset = get_preset("security_focused")
security_reviewer = CodeReviewer(config, **security_preset)

print("Security-focused reviewer created")
print(f"Criteria count: {len(security_reviewer.validation_criteria)}")

## Input/Output Format

**Input**: Code string (snippet, file, or diff)

**Output**:
```python
{
    "approved": bool,  # false if critical/high severity issues
    "issues": [
        {
            "severity": "critical|high|medium|low",
            "line": int | None,
            "category": "security|quality|style|performance",
            "message": str,
            "suggestion": str
        }
    ],
    "suggestions": [str],
    "summary": str,
    "timestamp": str
}
```

## Example: Basic Code Review

In [ ]:
# Sample code with issues
sample_code = '''
import os

def get_user_data(user_id):
    # Hardcoded credentials - security issue
    db_password = "super_secret_123"
    
    # SQL injection vulnerability
    query = f"SELECT * FROM users WHERE id = {user_id}"
    
    # No error handling
    connection = connect_to_db(db_password)
    result = connection.execute(query)
    
    return result
'''

print("Code to review:")
print(sample_code)

In [ ]:
# Run the review (uncomment to execute)
# result = await reviewer.run(sample_code)
# 
# print(f"Approved: {result.get('approved')}")
# print(f"\nIssues found: {len(result.get('issues', []))}")
# for issue in result.get('issues', []):
#     print(f"  [{issue['severity'].upper()}] {issue['category']}: {issue['message']}")
# 
# print(f"\nSummary: {result.get('summary')}")

## Example: Clean Code

In [ ]:
# Sample clean code
clean_code = '''
import os
from typing import Optional
import logging

logger = logging.getLogger(__name__)

def get_user_data(user_id: int) -> Optional[dict]:
    """Fetch user data from the database.
    
    Args:
        user_id: The unique identifier for the user.
        
    Returns:
        User data dictionary or None if not found.
        
    Raises:
        DatabaseError: If connection fails.
    """
    db_password = os.environ.get("DB_PASSWORD")
    if not db_password:
        logger.error("Database password not configured")
        raise EnvironmentError("DB_PASSWORD not set")
    
    try:
        connection = connect_to_db(db_password)
        # Parameterized query prevents SQL injection
        result = connection.execute(
            "SELECT * FROM users WHERE id = ?",
            (user_id,)
        )
        return result.fetchone()
    except DatabaseError as e:
        logger.exception("Database query failed")
        raise
    finally:
        connection.close()
'''

print("Clean code example:")
print(clean_code)

## Custom Configuration

You can customize the reviewer with your own criteria:

In [ ]:
# Custom reviewer with specific criteria
custom_reviewer = CodeReviewer(
    config=config,
    system_prompt="""You are a code reviewer focused on Python best practices.
    Focus on readability, maintainability, and Pythonic idioms.
    Be constructive and provide specific improvement suggestions.""",
    validation_criteria=[
        "Functions should have docstrings",
        "Use type hints for function parameters and returns",
        "Follow PEP 8 naming conventions",
        "Prefer list comprehensions over explicit loops when appropriate",
        "Use context managers for resource handling",
    ]
)

print("Custom reviewer created with criteria:")
for i, c in enumerate(custom_reviewer.validation_criteria, 1):
    print(f"  {i}. {c}")

## Severity Levels

Issues are categorized by severity:

| Severity | Description | Blocks Approval |
|----------|-------------|------------------|
| **critical** | Security vulnerabilities, data exposure | Yes |
| **high** | Bugs, logic errors, major quality issues | Yes |
| **medium** | Code smell, maintainability concerns | No |
| **low** | Style issues, minor suggestions | No |

## Next Steps

- **[04_pr_pipeline.ipynb](./04_pr_pipeline.ipynb)** - Multi-step PR review workflow
- **[05_release_pipeline.ipynb](./05_release_pipeline.ipynb)** - Automated release workflow
- **[00_getting_started.ipynb](./00_getting_started.ipynb)** - Framework overview